# Exemplo de Segmentação utilizando Grounding Dino e Segment Anything Model

[Grounding Dino](https://github.com/IDEA-Research/GroundingDINO) -> efetua deteções de forma automática com base em prompts (palavras chave/frases)

[Segment Anything Model](https://github.com/facebookresearch/segment-anything) -> efetua segmentações de forma automática, neste caso, com base em bounding boxes

Efetuar o download de ambos os repositórios e colocar no Google Drive.

## **Aceder ao Google Drive**

In [ ]:
from google.colab import drive

drive.mount('/content/drive/') # nome da diretoria onde serão colocados os ficheiros do Google Drive -> /nome_da_pasta/MyDrive/

# os conteúdos do Google Drive têm de estar numa diretoria vazia

## Instalar Grounding Dino

In [ ]:
# clonar o repositório -> isto gera uma diretoria chamada "GroundingDino"
!git clone https://github.com/IDEA-Research/GroundingDINO.git

# entrar na diretoria
%cd /content/GroundingDINO/

# instalar dependências
!pip install -e .

## Instalar Segment Anything Model (SAM)

In [ ]:
# voltar à diretoria inicial
%cd /content/

# colar o repositório -> isto gera uma diretoria chamada "segment-anything"
!git clone https://github.com/facebookresearch/segment-anything.git

# entrar na diretoria
%cd /content/segment-anything/

# instalar dependências
!pip install -e .

## Efetuar download dos pesos do Grounding Dino


In [ ]:
%cd /content/

# criar uma nova diretoria
!mkdir grounding_dino_weights

# entrar na nova diretoria
%cd grounding_dino_weights

# efetuar o download dos pesos
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

# voltar à diretoria inicial

%cd /content/

# **Importante**

Depois de instalar os dois repositórios, fazer o seguinte:

- Selecionar "Tempo de execução", que fica abaixo do título do notebook;
- Selecionar "Reiniciar sessão";
- Avançar para as próximas células.

## Bibliotecas do Grounding Dino

In [ ]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
from groundingdino.util import box_ops
from PIL import Image # ler imagens

from google.colab.patches import cv2_imshow # visualização de imagens

import locale # evitar erros
locale.getpreferredencoding = lambda: "UTF-8"

## Utilização do modelo Grounding Dino

## **Carregar o modelo Grounding Dino**

### **Parâmetros do Grounding Dino**

O text prompt pode ser constituído por palavras ou frases.

In [ ]:
text_prompt = "" # prompt que vai ser utilizado para efetuar as deteções -> "." indica a procura dos diferentes objetos de forma individual
box_threshold = 0 # número mínimo de similaridade entre as bounding boxes
text_threshold = 0 # número mínimo de similaridade entre as bounding boxes

## **Aplicar o Grounding Dino sobre uma imagem**

## Resultados da deteção

### Imagem original

In [ ]:
Image.fromarray(imagem_source)

### Imagem com deteções

In [ ]:
annotated_frame = annotate(image_source = imagem_source, boxes = boxes, logits = logits, phrases = phrases)

Image.fromarray(annotated_frame)

## Utilização do SAM

## Bibliotecas do SAM

In [ ]:
# Geral
import argparse
import os
import copy
import supervision as sv
from PIL import Image, ImageDraw, ImageFont
import torch
from imutils import contours # para ordenar contornos
import cv2

# segment anything
from segment_anything import build_sam, SamPredictor
import numpy as np

## **Download do checkpoint do  SAM**

In [ ]:
# carregar checkpoint do modelo SAM
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

## Inicializar o modelo SAM

## **Aplicar o modelo SAM sobre a imagem**

## **Normalizar as bounding boxes obtidaas pelo Grounding Dino**

In [ ]:
# normalizar as bounding boxes obtidas pelo Grounding Dino
H, W, _ = imagem_source.shape
boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

## **Calcular as máscaras com base nas bounding boxes do Grounding Dino**

In [ ]:
# obter máscaras com base nas bounding boxes
transformed_boxes = sam_predictor.transform.apply_boxes_torch(boxes_xyxy, imagem_source.shape[:2]).to("cuda")

masks, _, _ = sam_predictor.predict_torch(colocar parâmetros)

## **Função que transforma os arrays com a informação das máscaras em imagens**

In [ ]:
# Obter apenas as máscaras (preto e branco)
def mask_image(mask):

    color = np.array([255/255, 255/255, 255/255, 1]) # cor da máscara -> branca

    tensor = torch.Tensor.cpu(mask) # carregar a informação da GPU para a memória

    h, w = mask.shape[-2:] # obter as dimensões da máscara

    mask_image = tensor.reshape(h, w, 1) * color.reshape(1, 1, -1) # criar uma máscara com base no tensor

    mask = Image.fromarray((mask_image.cpu().numpy() * 255).astype(np.uint8)).convert("L") # converter o array para uma imagem

    return mask

## **Criar uma máscara com todas as máscaras**


### **Obter os contornos presentes na máscara**

Utilizar biblioteca cv2.

## **Ordenar os contornos da esquerda para a direita**

Utilizar biblioteca imutils.

## **Extrair contornos**

Utilizar biblioteca cv2.

In [ ]:
ep = 3 # padding

crop_num = 0


resized_crop = cv2.resize(crop, (20, 20)) # redimensionar a imagem recortada

# gravar imagem
cv2.imwrite(os.path.join(caminho, f'crop{crop_num}.png'), resized_crop)

# **Classificação dos caracteres**

## **Bibliotecas**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

## **Carregar o modelo de classificação**

In [ ]:
input_shape = (20, 20, 3) # input que o modelo aceita
num_classes = 34 # número de classes
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E',
              'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T',
              'U', 'V', 'W', 'X', 'Y', 'Z'] # nomes das classes -> não existe classe para a letra "O" e a letra "I"

modelo = keras.Sequential(
    [
        layers.Dense(32, input_shape =input_shape),
        layers.Dense(64, activation = 'relu'),
        layers.Flatten(),
        layers.Dense(num_classes, activation = 'softmax'),
    ]
)

In [ ]:
modelo.summary()

## **Carregar pesos pré-treinados**

In [ ]:
caminho_pesos = "/content/best_weights.h5"
modelo.load_weights(caminho_pesos)

## **Classificar os caracteres**

In [ ]:
imagem = cv2.imread('/content/crop0.png')

cv2_imshow(imagem)

imagem = np.expand_dims(imagem, axis = 0) # para garantir que as dimensões são as corretas

previsao = modelo.predict(imagem) # classificação por parte do modelo

classe = classes[np.argmax(previsao)] # classe prevista

print("Classe previsa:", classe)

# **Guardar resultados**
Exemplo de como guardar texto num ficheiro.

In [ ]:
caminho_ficheiro_resultados = "/content/resultados.txt" # caminho onde será guardado o ficheiro

resultados_f = open(caminho_ficheiro_resultados, "a") # gera o ficheiro caso não exista

# adicionar resultado ao ficheiro dos resultados
resultados_f.write("22XV69" + '\n') # \n adiciona um parágrafo